In [1]:
print('Loading modules....')

import os, sys


import uproot
import numpy as np
import matplotlib.pyplot as plt
from qetpy.utils import lowpassfilter 
from qetpy import autocuts
import cdms
#from CDMSDataCatalog import CDMSDataCatalog
import scipy.optimize as optimize
import matplotlib as mpl
import pickle as pkl
import ROOT as rt
import seaborn as sns
import operator as op
import pandas as pd
from collections import Counter

print('modules loaded...')

Loading modules....


/usr/local/lib/python3.10/dist-packages/CDMSDataCatalog/CDMSDataCatalog.py:6: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  import pkg_resources
/usr/local/lib/python3.10/dist-packages/pkg_resources/__init__.py:2976: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('sphinxcontrib')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  declare_namespace(pkg)


Welcome to JupyROOT 6.28/10
modules loaded...


In [10]:
#dataset = 'v2.1.1'
dataset = 'v3.0.0'
detNum = 3

In [11]:
# prod='CUTE_T3_GeActivation_%s_V05-00_P0.2.'%(dataset)
# base = '/fs/ddn/sdf/group/supercdms/data/CDMS/CUTE/R37/Processed/Tests/%s/Submerged'%(prod)

# if(dataset == 'prompt'):
#     base = '/fs/ddn/sdf/group/supercdms/data/CDMS/CUTE/R37/Processed/Prompt/Prompt_V04-15_P0.2.1/Submerged'
#     prod = 'Prompt_V04-15_P0.2.1'

prod='CUTE_T3_GeActivation_%s_V05-00_P0.2.'%(dataset)
base = '/fs/ddn/sdf/group/supercdms/data/CDMS/CUTE/R37/Processed/Tests/%s/Submerged'%(prod)

if(dataset == 'v3.0.0'):
    prod='CUTE_T3_Calibration_%s_V05-03_P0.3.3'%(dataset)
    base = '/fs/ddn/sdf/group/supercdms/data/CDMS/CUTE/R37/Processed/Tests/%s/Submerged'%(prod)

# if(dataset == 'v2.1.1'):
#     base = '/sdf/home/y/yanliu/data/cute'
#     prod='CUTE_T3_GeActivation_%s_V05-00_P0.2'%(dataset)

file_prefix = prod    
# series = ['23231221_015705','23231220_190923','23231220_122140','23231220_053358','23231220_012745',
#          '23231219_184002','23231219_110331','23231219_034952','23231218_223530','23231218_190035',
#          '23231218_152721','23231218_093255','23231217_212512','23231217_171613','23231217_135018']

#Found series in reprocessed data
# series = ['23231220_012745', '23231219_184002', '23231219_110331', '23231219_034952', 
#           '23231218_223530', '23231218_190035', '23231218_152721', '23231217_212512']

# only 4 series
#series = ['23231219_034952','23231219_110331','23231218_223530','23231217_212512']

# series = [
#     '23231220_012745','23231219_184002','23231219_110331','23231219_034952','23231218_223530',
#     '23231218_190035','23231218_152721','23231218_093255','23231217_212512','23231217_171613',
#     '23231217_135018'
#          ]
series = [
    '23231216_013604', '23231216_043946', '23231216_100125', '23231216_115012', '23231216_145300',
    '23231216_182937', '23231216_194929', '23231216_211119', '23231216_233807', '23231217_135018',
    '23231217_171613', '23231217_212512', '23231218_093255', '23231218_152721', '23231218_190035',
    '23231218_223530', '23231219_034952', '23231219_110331', '23231219_184002', '23231220_012745'
]
filePath = []
found_series = []
for ser in series:
    
    fp = '%s/%s_%s.root'%(base,file_prefix,ser)
    if(os.path.exists(fp)):
        print(ser,' : ',fp)
        filePath.append(fp)
        found_series.append(ser)
    else:
        print('not found: ',fp)
series = found_series
print('Found series = ',series)


print('Loading RQ and Event Tree...')
RQ=[]
for i,ser in enumerate(series):
    RQ.append(uproot.open(filePath[i])['rqDir/zip%d'%(detNum)]) 

eventTree=[]
for i,ser in enumerate(series):
    eventTree.append(uproot.open(filePath[i])['rqDir/eventTree']) 

print('done..')


print('Loading OF spefic data....')
#Load RQs from processed files
channels=['PT','PAS1','PBS1','PCS1','PDS1','PES1','PFS1','PAS2','PBS2','PCS2','PDS2','PES2','PFS2']

OFchiq={}
OFamps={}
OFdelay={}
OFamps0={}
OFnorms={}
OFbs={}

for i,key in enumerate(series):
    tc={}
    ta={}
    td={}
    ta0={}
    tn = {}
    tbs = {}
    for c in channels:
        if(detNum == 3 and c == 'PAS1'):
            print('\tSkipping Z3-PAS1...')
            continue
        # tc[c]=(RQ[i].array(c+"OFchisq"))
        # ta[c]=(RQ[i].array(c+"OFamps"))
        # ta0[c]=(RQ[i].array(c+"OFamps0"))
        # td[c]=(RQ[i].array(c+"OFdelay"))
        # if(c!='PT'):
        #     tn[c] = (RQ[i].array(c+"norm"))
        #     tbs[c] = (RQ[i].array(c+"bs"))
        tc[c]=(RQ[i][c+"OFchisq"].array(library='np'))
        ta[c]=(RQ[i][c+"OFamps"].array(library='np'))
        ta0[c]=(RQ[i][c+"OFamps0"].array(library='np'))
        td[c]=(RQ[i][c+"OFdelay"].array(library='np'))
        
        # tc[c]=(RQ[i].array(c+"OFchisq"))
        # ta[c]=(RQ[i].array(c+"OFamps"))
        # ta0[c]=(RQ[i].array(c+"OFamps0"))
        # td[c]=(RQ[i].array(c+"OFdelay"))
        if(c!='PT'):
            tn[c] = (RQ[i][c+"norm"].array(library='np'))
            tbs[c] = (RQ[i][c+"bs"].array(library='np'))
            # tn[c] = (RQ[i].array(c+"norm"))
            # tbs[c] = (RQ[i].array(c+"bs"))
    OFchiq[key]=tc
    OFamps[key]=ta
    OFamps0[key]=ta0
    OFdelay[key]=td
    OFnorms[key]=tn
    OFbs[key]=tbs
    print('%s loaded'%(key))
    
print('done!')

print(' Loading Event specific data....')


TriggerTime=[]
TriggerType=[]
EventNumber=[]
TrigDet=[]
TriggerMask=[]
EventTime=[]

# R2 = {} # Radial parameter for S2
# R1 = {} # Radial parameter for S1
# amp_diff = {}  
# amp_diff_abs = {}
for i,key in enumerate(series):
    # TriggerTime.append(eventTree[i].array("TriggerTime"))
    # TriggerType.append(eventTree[i].array("TriggerType"))
    # EventNumber.append(eventTree[i].array("EventNumber"))
    # TrigDet.append(eventTree[i].array("TriggerDetectorNum"))
    # TriggerMask.append(eventTree[i].array("TriggerMask"))
    # EventTime.append(eventTree[i].array("EventTime"))

    TriggerTime.append(eventTree[i]["TriggerTime"].array())
    TriggerType.append(eventTree[i]["TriggerType"].array())
    EventNumber.append(eventTree[i]["EventNumber"].array())
    TrigDet.append(eventTree[i]["TriggerDetectorNum"].array())
    TriggerMask.append(eventTree[i]["TriggerMask"].array())
    EventTime.append(eventTree[i]["EventTime"].array())
    
    # R2[key] = OFdelay[key]['PFS2'] - OFdelay[key]['PAS2']
    # R1[key] = OFdelay[key]['PFS1'] - OFdelay[key]['PAS1']
    # amp_diff[key] = (OFamps[key]['PFS2'] - OFamps[key]['PAS2'])/(OFamps[key]['PFS2'] + OFamps[key]['PAS2'])
    # amp_diff_abs[key] = (OFamps[key]['PFS2'] - OFamps[key]['PAS2'])
    print('%s loaded'%(key))

print('Done!')

23231216_013604  :  /fs/ddn/sdf/group/supercdms/data/CDMS/CUTE/R37/Processed/Tests/CUTE_T3_Calibration_v3.0.0_V05-03_P0.3.3/Submerged/CUTE_T3_Calibration_v3.0.0_V05-03_P0.3.3_23231216_013604.root
23231216_043946  :  /fs/ddn/sdf/group/supercdms/data/CDMS/CUTE/R37/Processed/Tests/CUTE_T3_Calibration_v3.0.0_V05-03_P0.3.3/Submerged/CUTE_T3_Calibration_v3.0.0_V05-03_P0.3.3_23231216_043946.root
23231216_100125  :  /fs/ddn/sdf/group/supercdms/data/CDMS/CUTE/R37/Processed/Tests/CUTE_T3_Calibration_v3.0.0_V05-03_P0.3.3/Submerged/CUTE_T3_Calibration_v3.0.0_V05-03_P0.3.3_23231216_100125.root
23231216_115012  :  /fs/ddn/sdf/group/supercdms/data/CDMS/CUTE/R37/Processed/Tests/CUTE_T3_Calibration_v3.0.0_V05-03_P0.3.3/Submerged/CUTE_T3_Calibration_v3.0.0_V05-03_P0.3.3_23231216_115012.root
23231216_145300  :  /fs/ddn/sdf/group/supercdms/data/CDMS/CUTE/R37/Processed/Tests/CUTE_T3_Calibration_v3.0.0_V05-03_P0.3.3/Submerged/CUTE_T3_Calibration_v3.0.0_V05-03_P0.3.3_23231216_145300.root
23231216_182937  :  

In [12]:

factors = [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]
calib_chans = ["PFS1", "PCS1", "PDS1", "PBS1", "PES1", "PAS1", "PFS2", "PCS2", "PBS2", "PES2", "PDS2", "PAS2"] # in TES0-11 order
#calib_chans = ['PAS1','PBS1','PCS1','PDS1','PES1','PFS1','PAS2','PBS2','PCS2','PDS2','PES2','PFS2']
#factors = []

calib = {}
for i,c in enumerate(calib_chans):
    calib[c] = factors[i]
reweight = 1.0
								
#### new calib normalized ####
if(dataset == 'v2.1.1'): 

    calib['PAS1'] = 0.0000
    calib['PBS1'] = 1.3653
    calib['PCS1'] = 1.1027
    calib['PDS1'] = 1.2077
    calib['PES1'] = 0.9557
    calib['PFS1'] = 0.9925
    calib['PAS2'] = 1.4751
    calib['PBS2'] = 0.9835
    calib['PCS2'] = 0.6400
    calib['PDS2'] = 0.9411
    calib['PES2'] = 0.6588
    calib['PFS2'] = 0.6776
    reweight = 1.0
    
elif(dataset == 'v2.1.0'):
    calib['PAS1'] = 1.4779 
    calib['PBS1'] = 0.9182
    calib['PCS1'] = 0.8745
    calib['PDS1'] = 1.0057
    calib['PES1'] = 0.8308
    calib['PFS1'] = 0.6996
    calib['PAS2'] = 1.3755
    calib['PBS2'] = 1.0321
    calib['PCS2'] = 0.7297
    calib['PDS2'] = 1.2756
    calib['PES2'] = 1.0508
    calib['PFS2'] = 0.7297
    reweight = 1.0    
elif(dataset == 'v3.0.0'):
    calib['PAS1'] = 0.0 
    calib['PBS1'] = 1.3653
    calib['PCS1'] = 1.1027
    calib['PDS1'] = 1.2077
    calib['PES1'] = 0.9557
    calib['PFS1'] = 0.9925
    calib['PAS2'] = 1.4751
    calib['PBS2'] = 0.9835
    calib['PCS2'] = 0.6400
    calib['PDS2'] = 0.9411
    calib['PES2'] = 0.6588
    calib['PFS2'] = 0.6776
    reweight = 1.0    
    	


In [13]:

filename = 'reproc_%s_Z%d_20ser_check_norm.root'%(dataset,detNum)
if(filename == ''):
    filename = 'reproc_%s_Z%d.root'%(dataset,detNum)
    
outfile = rt.TFile.Open(filename,'recreate')
outfile.cd()


def deg(angle):
    return angle*(np.pi/180)


print('Calculating triangles......')
CDE_S1_X = {}
CDE_S1_Y = {}
CDE_S2_X = {}
CDE_S2_Y = {}
CDE_S1_X_corr = {}
CDE_S1_Y_corr = {}
CDE_S2_X_corr = {}
CDE_S2_Y_corr = {}

BDF_S1_X = {}
BDF_S1_Y = {}
BDF_S2_X = {}
BDF_S2_Y = {}
BDF_S1_X_corr = {}
BDF_S1_Y_corr = {}
BDF_S2_X_corr = {}
BDF_S2_Y_corr = {}

ADF_S1_X = {}
ADF_S1_Y = {}
ADF_S2_X = {}
ADF_S2_Y = {}
ADF_S1_X_corr = {}
ADF_S1_Y_corr = {}
ADF_S2_X_corr = {}
ADF_S2_Y_corr = {}

asym_par = {}
asym_par_corr = {}


for idx, key in enumerate(series):
    A1_def = 0.0 #OFamps[key]['PAS1']
    B1_def = OFamps[key]['PBS1']
    C1_def = OFamps[key]['PCS1']
    D1_def = OFamps[key]['PDS1']
    E1_def = OFamps[key]['PES1']
    F1_def = OFamps[key]['PFS1']

    A2_def = OFamps[key]['PAS2']
    B2_def = OFamps[key]['PBS2']
    C2_def = OFamps[key]['PCS2']
    D2_def = OFamps[key]['PDS2']
    E2_def = OFamps[key]['PES2']
    F2_def = OFamps[key]['PFS2']
    
    A1 = calib['PAS1'] * A1_def
    B1 = calib['PBS1'] * B1_def
    C1 = calib['PCS1'] * C1_def
    D1 = calib['PDS1'] * D1_def
    E1 = calib['PES1'] * E1_def
    F1 = calib['PFS1'] * F1_def
    
    A2 = calib['PAS2'] * A2_def
    B2 = calib['PBS2'] * B2_def
    C2 = calib['PCS2'] * C2_def
    D2 = calib['PDS2'] * D2_def
    E2 = calib['PES2'] * E2_def
    F2 = calib['PFS2'] * F2_def
    
    bdf_s1_x = (F1_def*np.cos(deg(0)) + B1_def*np.cos(deg(120)) + D1_def*np.cos(deg(240)))/(F1_def+B1_def+D1_def)
    bdf_s1_y = (F1_def*np.sin(deg(0)) + B1_def*np.sin(deg(120)) + D1_def*np.sin(deg(240)))/(F1_def+B1_def+D1_def)
    bdf_s2_x = (B2_def*np.cos(deg(0+60)) + F2_def*np.cos(deg(120+60)) + D2_def*np.cos(deg(240+60)))/(F2_def+B2_def+D2_def)
    bdf_s2_y = (B2_def*np.sin(deg(0+60)) + F2_def*np.sin(deg(120+60)) + D2_def*np.sin(deg(240+60)))/(F2_def+B2_def+D2_def)
    BDF_S1_X[key] = np.array(bdf_s1_x)
    BDF_S1_Y[key] = np.array(bdf_s1_y)
    BDF_S2_X[key] = np.array(bdf_s2_x)
    BDF_S2_Y[key] = np.array(bdf_s2_y)

    bdf_s1_x = (F1*np.cos(deg(0)) + B1*np.cos(deg(120)) + D1*np.cos(deg(240)))/(F1+B1+D1)
    bdf_s1_y = (F1*np.sin(deg(0)) + B1*np.sin(deg(120)) + D1*np.sin(deg(240)))/(F1+B1+D1)
    bdf_s2_x = (B2*np.cos(deg(0+60)) + F2*np.cos(deg(120+60)) + D2*np.cos(deg(240+60)))/(F2+B2+D2)
    bdf_s2_y = (B2*np.sin(deg(0+60)) + F2*np.sin(deg(120+60)) + D2*np.sin(deg(240+60)))/(F2+B2+D2)
    BDF_S1_X_corr[key] = np.array(bdf_s1_x)
    BDF_S1_Y_corr[key] = np.array(bdf_s1_y)
    BDF_S2_X_corr[key] = np.array(bdf_s2_x)
    BDF_S2_Y_corr[key] = np.array(bdf_s2_y)
    
    
    cde_s1_x = (E1_def*np.cos(deg(0)) + C1_def*np.cos(deg(120)) + D1_def*np.cos(deg(240)))/(E1_def+C1_def+D1_def)
    cde_s1_y = (E1_def*np.sin(deg(0)) + C1_def*np.sin(deg(120)) + D1_def*np.sin(deg(240)))/(E1_def+C1_def+D1_def)
    cde_s2_x = (C2_def*np.cos(deg(0+60)) + E2_def*np.cos(deg(120+60)) + D2_def*np.cos(deg(240+60)))/(E2_def+C2_def+D2_def)
    cde_s2_y = (C2_def*np.sin(deg(0+60)) + E2_def*np.sin(deg(120+60)) + D2_def*np.sin(deg(240+60)))/(E2_def+C2_def+D2_def)
    CDE_S1_X[key] = np.array(cde_s1_x)
    CDE_S1_Y[key] = np.array(cde_s1_y)
    CDE_S2_X[key] = np.array(cde_s2_x)
    CDE_S2_Y[key] = np.array(cde_s2_y)

    cde_s1_x = (E1*np.cos(deg(0)) + C1*np.cos(deg(120)) + D1*np.cos(deg(240)))/(E1+C1+D1)
    cde_s1_y = (E1*np.sin(deg(0)) + C1*np.sin(deg(120)) + D1*np.sin(deg(240)))/(E1+C1+D1)
    cde_s2_x = (C2*np.cos(deg(0+60)) + E2*np.cos(deg(120+60)) + D2*np.cos(deg(240+60)))/(E2+C2+D2)
    cde_s2_y = (C2*np.sin(deg(0+60)) + E2*np.sin(deg(120+60)) + D2*np.sin(deg(240+60)))/(E2+C2+D2)
    CDE_S1_X_corr[key] = np.array(cde_s1_x)
    CDE_S1_Y_corr[key] = np.array(cde_s1_y)
    CDE_S2_X_corr[key] = np.array(cde_s2_x)
    CDE_S2_Y_corr[key] = np.array(cde_s2_y)
    
    adf_s1_x = (F1_def*np.cos(deg(0)) + A1_def*np.cos(deg(120)) + D1_def*np.cos(deg(240)))/(F1_def+A1_def+D1_def)
    adf_s1_y = (F1_def*np.sin(deg(0)) + A1_def*np.sin(deg(120)) + D1_def*np.sin(deg(240)))/(F1_def+A1_def+D1_def)
    adf_s2_x = (A2_def*np.cos(deg(0+60)) + F2_def*np.cos(deg(120+60)) + D2_def*np.cos(deg(240+60)))/(F2_def+A2_def+D2_def)
    adf_s2_y = (A2_def*np.sin(deg(0+60)) + F2_def*np.sin(deg(120+60)) + D2_def*np.sin(deg(240+60)))/(F2_def+A2_def+D2_def)
    ADF_S1_X[key] = np.array(adf_s1_x)
    ADF_S1_Y[key] = np.array(adf_s1_y)
    ADF_S2_X[key] = np.array(adf_s2_x)
    ADF_S2_Y[key] = np.array(adf_s2_y)

    adf_s1_x = (F1*np.cos(deg(0)) + A1*np.cos(deg(120)) + D1*np.cos(deg(240)))/(F1+A1+D1)
    adf_s1_y = (F1*np.sin(deg(0)) + A1*np.sin(deg(120)) + D1*np.sin(deg(240)))/(F1+A1+D1)
    adf_s2_x = (A2*np.cos(deg(0+60)) + F2*np.cos(deg(120+60)) + D2*np.cos(deg(240+60)))/(F2+A2+D2)
    adf_s2_y = (A2*np.sin(deg(0+60)) + F2*np.sin(deg(120+60)) + D2*np.sin(deg(240+60)))/(F2+A2+D2)
    ADF_S1_X_corr[key] = np.array(adf_s1_x)
    ADF_S1_Y_corr[key] = np.array(adf_s1_y)
    ADF_S2_X_corr[key] = np.array(adf_s2_x)
    ADF_S2_Y_corr[key] = np.array(adf_s2_y)
    
    #s1 = A1_def + B1_def + C1_def + D1_def + E1_def + F1_def
    s1 = B1_def + C1_def + D1_def + E1_def + F1_def
    s2 = A2_def + B2_def + C2_def + D2_def + E2_def + F2_def
    asym_par[key] = (s1 - s2)/(s1 + s2)
    s1 = A1 + B1 + C1 + D1 + E1 + F1
    s1 = s1*reweight
    s2 = A2 + B2 + C2 + D2 + E2 + F2
    
    asym_par_corr[key] = (s1 - s2)/(s1 + s2)
    
    print('%s done'%(key))
    
    
def make_graph(x,y,name):
    temp = rt.TGraph(len(x),x,y)
    temp.SetName(name)
    return(temp)

def make_hist(y,name):
    temp_hist = rt.TH1F(name,name,200,-0.3,0.3)
    #temp_hist = rt.TH1F(name,name,200,-1.0,1.0)
    for i in range(len(y)):
        temp_hist.Fill(y[i])
    return temp_hist


def compute_triangle(X,Y,low,high):
    x = np.array([])
    y = np.array([])
    chan = 'PT'
    for idx, key in enumerate(series):
        #mask = (TriggerType[idx] == 1) & (TrigDet[idx] == 1) & (OFamps[key][chan] > 25e-6) & (OFamps[key][chan] < 32e-6) & (OFchiq[key][chan] < 5e5)
        mask = (TriggerType[idx] == 1) & (TrigDet[idx] == detNum) & (OFamps[key][chan] > low) & (OFamps[key][chan] < high) & (OFchiq[key][chan] < 5e5) & (abs(OFdelay[key][chan])*1e3 < 20)
        x = np.append(x,X[key][mask])
        y = np.append(y,Y[key][mask])
    return(x,y)


def compute_Z(Y,low,high):
    x = np.array([])
    y = np.array([])
    chan = 'PT'
    for idx, key in enumerate(series):
        #mask = (TriggerType[idx] == 1) & (TrigDet[idx] == 1) & (OFamps[key][chan] > 25e-6) & (OFamps[key][chan] < 32e-6) & (OFchiq[key][chan] < 5e5)
        mask = (TriggerType[idx] == 1) & (TrigDet[idx] == detNum) & (OFamps[key][chan] > low) & (OFamps[key][chan] < high) & (OFchiq[key][chan] < 5e5) & (abs(OFdelay[key][chan])*1e3 < 20)
        x = np.append(x,OFamps[key][chan][mask])
        y = np.append(y,Y[key][mask])
    return(x,y)



graphs = []
hists = []

'''
Z3
160 eV=[0.7uA, 1.1uA]
1.3 keV= [7uA, 9uA]
10 keV= [50uA, 80uA]
'''
PT160ev=[0.75e-6, 1.2e-6]
PT1kev= [4e-6, 8e-6]
PT10kev= [40e-6, 50e-6]

'''
 Make CDE graph
'''

x,y = compute_triangle(CDE_S1_X,CDE_S1_Y,PT1kev[0],PT1kev[1])
graphs.append(make_graph(x,y,'CDE_S1_1kev'))

x,y = compute_triangle(CDE_S2_X,CDE_S2_Y,PT1kev[0],PT1kev[1])
graphs.append(make_graph(x,y,'CDE_S2_1kev'))

x,y = compute_triangle(CDE_S1_X_corr,CDE_S1_Y_corr,PT1kev[0],PT1kev[1])
graphs.append(make_graph(x,y,'CDE_S1_1kev_corr'))

x,y = compute_triangle(CDE_S2_X_corr,CDE_S2_Y_corr,PT1kev[0],PT1kev[1])
graphs.append(make_graph(x,y,'CDE_S2_1kev_corr'))


x,y = compute_triangle(CDE_S1_X,CDE_S1_Y,PT10kev[0],PT10kev[1])
graphs.append(make_graph(x,y,'CDE_S1_10kev'))

x,y = compute_triangle(CDE_S2_X,CDE_S2_Y,PT10kev[0],PT10kev[1])
graphs.append(make_graph(x,y,'CDE_S2_10kev'))

x,y = compute_triangle(CDE_S1_X_corr,CDE_S1_Y_corr,PT10kev[0],PT10kev[1])
graphs.append(make_graph(x,y,'CDE_S1_10kev_corr'))

x,y = compute_triangle(CDE_S2_X_corr,CDE_S2_Y_corr,PT10kev[0],PT10kev[1])
graphs.append(make_graph(x,y,'CDE_S2_10kev_corr'))

print('CDE graphs computed....')
'''
 Make BDF graph
'''

x,y = compute_triangle(BDF_S1_X,BDF_S1_Y,PT1kev[0],PT1kev[1])
graphs.append(make_graph(x,y,'BDF_S1_1kev'))

x,y = compute_triangle(BDF_S2_X,BDF_S2_Y,PT1kev[0],PT1kev[1])
graphs.append(make_graph(x,y,'BDF_S2_1kev'))

x,y = compute_triangle(BDF_S1_X_corr,BDF_S1_Y_corr,PT1kev[0],PT1kev[1])
graphs.append(make_graph(x,y,'BDF_S1_1kev_corr'))

x,y = compute_triangle(BDF_S2_X_corr,BDF_S2_Y_corr,PT1kev[0],PT1kev[1])
graphs.append(make_graph(x,y,'BDF_S2_1kev_corr'))


x,y = compute_triangle(BDF_S1_X,BDF_S1_Y,PT10kev[0],PT10kev[1])
graphs.append(make_graph(x,y,'BDF_S1_10kev'))

x,y = compute_triangle(BDF_S2_X,BDF_S2_Y,PT10kev[0],PT10kev[1])
graphs.append(make_graph(x,y,'BDF_S2_10kev'))

x,y = compute_triangle(BDF_S1_X_corr,BDF_S1_Y_corr,PT10kev[0],PT10kev[1])
graphs.append(make_graph(x,y,'BDF_S1_10kev_corr'))

x,y = compute_triangle(BDF_S2_X_corr,BDF_S2_Y_corr,PT10kev[0],PT10kev[1])
graphs.append(make_graph(x,y,'BDF_S2_10kev_corr'))

print('BDF graphs computed....')


'''
 Make ADF graph
'''

x,y = compute_triangle(ADF_S1_X,ADF_S1_Y,PT1kev[0],PT1kev[1])
graphs.append(make_graph(x,y,'ADF_S1_1kev'))

x,y = compute_triangle(ADF_S2_X,ADF_S2_Y,PT1kev[0],PT1kev[1])
graphs.append(make_graph(x,y,'ADF_S2_1kev'))

x,y = compute_triangle(ADF_S1_X_corr,ADF_S1_Y_corr,PT1kev[0],PT1kev[1])
graphs.append(make_graph(x,y,'ADF_S1_1kev_corr'))

x,y = compute_triangle(ADF_S2_X_corr,ADF_S2_Y_corr,PT1kev[0],PT1kev[1])
graphs.append(make_graph(x,y,'ADF_S2_1kev_corr'))


x,y = compute_triangle(ADF_S1_X,ADF_S1_Y,PT10kev[0],PT10kev[1])
graphs.append(make_graph(x,y,'ADF_S1_10kev'))

x,y = compute_triangle(ADF_S2_X,ADF_S2_Y,PT10kev[0],PT10kev[1])
graphs.append(make_graph(x,y,'ADF_S2_10kev'))

x,y = compute_triangle(ADF_S1_X_corr,ADF_S1_Y_corr,PT10kev[0],PT10kev[1])
graphs.append(make_graph(x,y,'ADF_S1_10kev_corr'))

x,y = compute_triangle(ADF_S2_X_corr,ADF_S2_Y_corr,PT10kev[0],PT10kev[1])
graphs.append(make_graph(x,y,'ADF_S2_10kev_corr'))

print('ADF graphs computed....')


x,y=compute_Z(asym_par,PT1kev[0],PT1kev[1])
graphs.append(make_graph(x,y,'Z_1kev')) 
hists.append(make_hist(y,'h_Z_1kev'))

x,y=compute_Z(asym_par_corr, PT1kev[0],PT1kev[1])
graphs.append(make_graph(x,y,'Z_1kev_corr')) 
hists.append(make_hist(y,'h_Z_1kev_corr'))

x,y=compute_Z(asym_par,PT10kev[0],PT10kev[1])
graphs.append(make_graph(x,y,'Z_10kev')) 
hists.append(make_hist(y,'h_Z_10kev'))

x,y=compute_Z(asym_par_corr, PT10kev[0],PT10kev[1])
graphs.append(make_graph(x,y,'Z_10kev_corr'))
hists.append(make_hist(y,'h_Z_10kev_corr'))

x,y=compute_Z(asym_par,0.4e-6, 60e-6)
graphs.append(make_graph(x,y,'Z_all')) 
hists.append(make_hist(y,'h_Z_all'))

x,y=compute_Z(asym_par_corr, 0.4e-6, 60e-6)
graphs.append(make_graph(x,y,'Z_all_corr'))
hists.append(make_hist(y,'h_Z_all_corr'))


x = np.array([])
y = np.array([])
chan = 'PT'
for idx, key in enumerate(series):
    mask = (TriggerType[idx] == 1) & (TrigDet[idx] == detNum) & (OFamps[key][chan] > 0)# & (OFchiq[key][chan] < 5e5)
    x = np.append(x,OFamps[key][chan][mask])
    y = np.append(y,OFchiq[key][chan][mask])

h_chiq_PTOF = rt.TH2F('h_chiq_PTOF','h_chiq_PTOF',100,-8, -3,100,np.logspace(4.3, 9, 101)) 
h_chiq_PTOF.GetXaxis().SetTitle('PTOFamps [uA]')
h_chiq_PTOF.GetYaxis().SetTitle('Chiq')
h_chiq_PTOF.SetStats(0)
for i in range(len(x)):
    h_chiq_PTOF.Fill(np.log10(x[i]),y[i])
    





for g in graphs:
    g.Write()
    
outfile.Write()
outfile.Close()
print('%s file written..'%(filename))
print('----- All done ----')
print('Use %s as input in plotting script...'%(filename))

Calculating triangles......
23231216_013604 done
23231216_043946 done
23231216_100125 done
23231216_115012 done
23231216_145300 done
23231216_182937 done
23231216_194929 done
23231216_211119 done
23231216_233807 done
23231217_135018 done
23231217_171613 done
23231217_212512 done
23231218_093255 done
23231218_152721 done
23231218_190035 done
23231218_223530 done
23231219_034952 done
23231219_110331 done
23231219_184002 done
23231220_012745 done
CDE graphs computed....
BDF graphs computed....
ADF graphs computed....
reproc_v3.0.0_Z3_20ser_check_norm.root file written..
----- All done ----


## Obtain normalized calibration values.
(Run it when the calibrations are finalized)